[run in colab](https://colab.research.google.com/github/raoulg/tmoi-ml-20/blob/master/4-tuning_dnn/tuning_dnn.ipynb)

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/raoulg/tmoi-ml-20/blob/master/4-tuning_dnn/tuning_dnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
import tensorflow as tf 
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist, mnist
import datetime
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import matplotlib.pyplot as plt

In [ ]:
# When running on a location without the utils.py file, download it.
files = !ls
if 'utils.py' not in files:
    file = "https://raw.githubusercontent.com/raoulg/tmoi-ml-20/master/4-tuning_dnn/utils.py"
    req = requests.get(file)
    url_content = req.content
    csv_file = open('utils.py', 'wb')
    csv_file.write(url_content)
    csv_file.close()
from utils import *

In [ ]:
# install some missing libraries if running on colab
# notify the user if GPU is off

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

if IS_COLAB:
    !pip install keras-tuner
    !pip install tensorflow-addons

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Load the data
The MNIST sort of became too easy. We get levels close to 99%, so it get's increasingly harder to see what improves or not. So we want a set that is harder. The fashion_mnist is a bit harder, but still doable with relative easy techniques. We can still train on a default laptop, while it is hard enough to have some space for tweaking and improving.

In [ ]:
(X_train, y_train), (X_valid, y_valid) = fashion_mnist.load_data()
from sklearn.model_selection import train_test_split
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.4, random_state=42)
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
X_valid.shape, y_valid.shape

In [ ]:
# labels are sparse
y_valid

Let's explore the set a bit.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

# Baseline
Build a baseline deep learning model with settings you guess are simple, but good enough to give you a baseline to improve. E.g. try just one or two layers of Dense layers.

Store results and scores.

If you are curious about simple (non-deep learning models) You can have a look here [here](http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/#) for benchmarks with simple models. Note the trainingtime, before you attempt to reproduce this with a simple model; some really take hours! Also try to sort the models by accuracy, and note the best model: an SVC with a kernel... Downside is, that it takes more than an hour to run.

In [ ]:
result = {}
score = {}

# Dropout, Batchnorm, activations

Experiment manually with batchnorm, drops and different activations to get a sense of how they impact the model. 
Store your results in dictionaries, such that is it easy to compare (see lesson for examples). If you want to test with `Conv2D` and `MaxPool2D` layers, you can use `Reshape((28,28,1))` to get the desired 4D shape for image convolutions.

Getting above 90% on the testset is actually pretty good for the fashion mnist.
 

In [ ]:
X_train.shape

In [ ]:
rs1 = X_train.reshape((60000, 28, 28, 1))
rs1.shape

In [ ]:
from tensorflow.keras.layers import Reshape
x = X_train[:32]
rs3 = Reshape((28,28,1))(x)
rs3.shape, x.shape

In [ ]:
# your models here

In [ ]:
plot_results(result)

In [ ]:
plot_scores(score)

Save your best manual tuned model.

In [ ]:
model.save('manual_tune.h5')

# Hypertuning
Build a hypermodel. Start with broad settings, and use `hp.Fixed` to fixate things your are fairly sure about. You should be able to get above 90% accuracy on the testset.

In [ ]:
import kerastuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=5, # increasing the amount of epochs will increase the amount of trials
    factor=3, # decreasing the factor will increase the amount of trials
    seed=10, # by setting the seed, you guarantee the same outcome every time you run the hyperband
    hyperband_iterations=1, # run the complete algorithm more than once, starting from scratch every time.
    directory='ktuner',
    project_name='fashion_mnist'
)
tuner.search(X_train, y_train, validation_data = (X_valid, y_valid), verbose=1)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print(best_hps.values)
model = tuner.get_best_models()[0]

In [ ]:
%%time 
result['hyper'] = model.fit(X_train, y_train, 
    epochs=100,
    validation_data=(X_valid, y_valid), 
    callbacks=[early_stop],
    verbose = 1)

In [ ]:
plot_results(result)

In [ ]:
score['hyper']=model.evaluate(X_test, y_test)

In [ ]:
plot_scores(score, ymin=0.8)

In [ ]:
model.save('hypertune.h5')

In [ ]:
model = load_model('hypertune.h5')
model.summary()